In [1]:
import findspark
findspark.init()

In [2]:
import time
import os

----

In [3]:
import numpy as np
from scipy import sparse

def get_vals(w: int = 5):
    coefs = np.random.randint(1, 1000, (w,w))
    p = np.random.randint(3000, 100000, w)
    return coefs, p + ~(p % 2).astype(bool)

def fit_poly(row, x):
    return np.poly1d(row)(x)

def cm_multihash(x: int, counts: int, coefs, p, const: int = 2719):
    table = np.zeros((p.shape[0], const))
    j = ((np.apply_along_axis(fit_poly, 1, coefs, x) % p) % const).astype(int)
    table[np.arange(0, p.shape[0]),j] = counts
    return sparse.csr_matrix(table)

def sparse_sim(sk1: sparse.csr.csr_matrix, sk2: sparse.csr.csr_matrix):
    return sk1.multiply(sk2).sum(axis = 1).min()

In [4]:
coefs, p = get_vals()

----

In [41]:
tau = 200

In [42]:
start_time = time.process_time()
collect_hashes = lines.window(60)\
                        .map(lambda line: line.split(','))\
                        .map(lambda line: (line[0], cm_multihash(int(line[1]), 1, coefs, p)))\
                        .reduceByKey(lambda x,y: x+y)

In [43]:
prepared = collect_hashes.map(lambda line: (1, [line[0], line[1]]))
combo = prepared.transform(lambda RDD: RDD.join(RDD).filter(lambda row: row[1][0][0] > row[1][1][0])\
                                          .map(lambda row: [row[1][0][1], row[1][1][1]]))\
                .map(lambda line: sparse_sim(line[-2], line[-1]))\
                .filter(lambda line: line[-1] > tau).count()\
                #.map(lambda line: [line, (time.process_time() - start_time)])
                
combo.pprint(5)

WORKING NOW: 10 rows per second 

In [44]:
ssc.start()

In [45]:
ssc.stop()

------

In [11]:
os.getcwd()+'\\checkpoint_dir'

'C:\\Users\\onii-chan\\Documents\\JupyterNotebooks\\Big Data Management\\Milestone 2\\checkpoint_dit'

In [7]:
import pyspark
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

sc = SparkContext() 
#ss = SparkSession.builder.appName("s1").getOrCreate()
#lines = ss.readStream.format("socket").option("host", "localhost").option("port",9000).load()
ssc = StreamingContext(sc, 2)
lines = ssc.socketTextStream("localhost", 9000)

#os.mkdir('checkpoint_dir')
ssc.checkpoint(os.getcwd()+'\\checkpoint_dir')

C:\Spark\spark-3.2.0-bin-hadoop3.2\python\pyspark\context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [10]:
def tmp_fun(a,b):
    return a+b

def inv_fun(a,b):
    return a-b

accum = sc.accumulator(0)

collect_hashes = lines.foreachRDD(lambda rdd: rdd.count())\
                      .map(lambda line: line.split(','))\
                      .map(lambda line: (line[0], cm_multihash(int(line[1]), 1, coefs, p)))\
                      .reduceByKeyAndWindow(tmp_fun, inv_fun, 60, 2)

SyntaxError: lambda cannot contain assignment (<ipython-input-10-3c1ff46a5bd9>, line 9)

In [14]:
prepared = collect_hashes.map(lambda line: (1, [line[0], line[1]]))
combo = prepared.transform(lambda RDD: RDD.join(RDD).map(lambda row: [row[1][0][0], row[1][1][0],
                                                                      row[1][0][1], row[1][1][1]]))\
                .filter(lambda line: line[0] > line[1])\
                .map(lambda line: [line[0], line[1], sparse_sim(line[-2], line[-1])])\
                .count()
                #.filter(lambda line: line[-1] > tau)\
                
combo.pprint(5)

In [15]:
ssc.start()

-------------------------------------------
Time: 2022-03-25 22:49:32
-------------------------------------------
1891



In [13]:
ssc.stop()

----

### The code below is WORKING code

In [5]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext


sc = SparkContext() 
ssc = StreamingContext(sc, 2)
lines = ssc.socketTextStream("localhost", 9000)

#os.mkdir('checkpoint_dir')
ssc.checkpoint(os.getcwd()+'\\checkpoint_dir')

C:\Spark\spark-3.2.0-bin-hadoop3.2\python\pyspark\context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [6]:
tau = 200

In [7]:
def getSparkSessionInstance(sparkConf):
    if ("sparkSessionSingletonInstance" not in globals()):
        globals()["sparkSessionSingletonInstance"] = SparkSession \
            .builder\
            .getOrCreate()
    return globals()["sparkSessionSingletonInstance"]

In [8]:
def getUpdatesCounter(sparkContext):
    if ("counter" not in globals()):
        globals()["counter"] = sparkContext.accumulator(0)
    return globals()["counter"]

In [9]:
def tmp_fun(a,b):
    return a+b

def inv_fun(a,b):
    return a-b

In [10]:
def counting(time, rdd):
    counter = getUpdatesCounter(rdd.context)
    counter.add(rdd.count())

In [11]:
def process_data(time, rdd):
    #print("========= %s =========" % str(time))
    counter = getUpdatesCounter(rdd.context)
    #counter.add(rdd.count())
    collect = rdd.map(lambda line: line.split(','))\
                 .map(lambda line: (line[0], cm_multihash(int(line[1]), 1, coefs, p)))\
                 .reduceByKey(tmp_fun)\
                 .map(lambda line: (1, [line[0], line[1]]))
    fin = collect.join(collect).map(lambda row: [row[1][0][0], row[1][1][0], row[1][0][1], row[1][1][1]])\
                 .filter(lambda line: line[0] > line[1])\
                 .map(lambda line: [line[0], line[1], sparse_sim(line[-2], line[-1])])\
                 .filter(lambda line: line[2] > tau).count()
    print(f'>>>> {str(time)[-8:]}  =====  {fin}  =====  {counter.value}')
    

In [12]:
lines.foreachRDD(counting)
lines.window(60).foreachRDD(process_data)

In [13]:
ssc.start()

>>>> 22:03:58  =====  0  =====  100
>>>> 22:04:00  =====  0  =====  300
>>>> 22:04:02  =====  0  =====  400
>>>> 22:04:04  =====  0  =====  600
>>>> 22:04:06  =====  0  =====  800
>>>> 22:04:08  =====  0  =====  1000


In [ ]:
ssc.stop()